<a href="https://colab.research.google.com/github/OlgaNezh/NLP/blob/main/NLP_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Урок 4. Тематическое моделирование. EM-алгоритм
**Задача тематическое моделирование**

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

рабочие ссылки с твитами
https://disk.yandex.ru/i/v5HM-ENiGXZVpQ
https://disk.yandex.ru/i/koR5eMCToCZS2Q

как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук parse_rospotrebnadzor.ipynb
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики



In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 8.2 MB 14.6 MB/s 


In [4]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 9.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=2825454f3cd947731c03702ef2b822a68f45dec13adb53b01ccc03bde050a573
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [5]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from pymystem3 import Mystem
import nltk
nltk.download('stopwords')
import pymorphy2
from gensim.models import *
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


**1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов**


In [7]:
positive = pd.read_csv('/content/drive/MyDrive/lessons4/positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('/content/drive/MyDrive/lessons4/negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = pd.concat([positive, negative], ignore_index=True)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [9]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens = regex.findall(text) 
    return [w for w in tokens if w.isalpha() and len(w)>= 4] 

morph = pymorphy2.MorphAnalyzer() 
stopwords_list = stopwords.words('russian') 

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] 
    return [w for w in lemmas if not w in stopwords and w.isalpha()] 

def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    string_tokens = re.sub(r'[A-za-z]', ' ',  string_tokens) 
    string_tokens = re.sub(r'\w+ще\b', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bбл\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'значит', ' ',  string_tokens) 
    string_tokens = re.sub(r'почему', ' ',  string_tokens) 
    string_tokens = re.sub(r'свой', ' ',  string_tokens) 
    string_tokens = re.sub(r'капец', ' ',  string_tokens) 
    string_tokens = re.sub(r'который', ' ',  string_tokens)
    string_tokens = re.sub(r'это', ' ',  string_tokens)
    string_tokens = re.sub(r'весь', ' ',  string_tokens)
    string_tokens = re.sub(r'\bидио\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bпид\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхуй\w+', ' ', string_tokens)
    string_tokens = re.sub(r'\w+xу.\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+xу\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+но\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпоп\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпиз\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bгов\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bфиг\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'сука', ' ',  string_tokens)
    string_tokens = re.sub(r'очень', ' ',  string_tokens)
    string_tokens = re.sub(r'лох', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпи.де\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'нету', ' ',  string_tokens)
    string_tokens = re.sub(r'ой', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхот\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпрост\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bника\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ть\b', ' ',  string_tokens)
    string_tokens = re.sub(r'мочь', ' ',  string_tokens)
    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: DeprecationWarning: invalid escape sequence \w
<input>:1: Dep

In [10]:
df['text'] = df['text'].apply(preprocess)
df.head()

,text,label
0,"[школотый, поверь, самый, общество, предмет, тип]",positive
1,"[таки, немного, похожий, мальчик, хороший]",positive
2,[испугаться],positive
3,"[угол, голод, порция]",positive
4,"[страшилка, создаться, ощущение, автор]",positive


**2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать**


In [11]:
dictionary = corpora.Dictionary(df['text']) 

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) 

dictionary.save('tweet.dict')

len(dictionary.keys()) 

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


8172

In [12]:
corpus = [dictionary.doc2bow(text) for text in df['text']] 
corpora.MmCorpus.serialize('tweet.model', corpus) 

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


In [13]:
mm = corpora.MmCorpus('tweet.model')
print(df['text'].values[0])
print(mm[0])

['школотый', 'поверь', 'самый', 'общество', 'предмет', 'тип']
[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]


/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:494: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


In [14]:
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sc

In [15]:
lda.show_topics(num_topics=3, num_words=20, formatted=True)

[(0,
  '0.040*"день" + 0.021*"хороший" + 0.020*"больш" + 0.017*"жизнь" + 0.017*"человек" + 0.014*"самый" + 0.011*"любимый" + 0.009*"скоро" + 0.009*"экзамен" + 0.009*"наш" + 0.008*"каникулы" + 0.008*"фильм" + 0.007*"боль" + 0.007*"первый" + 0.007*"спасибо" + 0.006*"найти" + 0.006*"друг" + 0.006*"город" + 0.006*"правда" + 0.006*"конец"'),
 (1,
  '0.038*"сегодня" + 0.013*"жаль" + 0.012*"час" + 0.010*"урок" + 0.010*"ночь" + 0.010*"голова" + 0.008*"вчера" + 0.008*"телефон" + 0.007*"дом" + 0.007*"жалко" + 0.007*"бояться" + 0.007*"вроде" + 0.007*"друг" + 0.006*"человек" + 0.006*"мало" + 0.006*"минута" + 0.006*"вечер" + 0.006*"ужасный" + 0.006*"фотка" + 0.006*"чувство"'),
 (2,
  '0.029*"завтра" + 0.016*"школа" + 0.016*"время" + 0.015*"новый" + 0.013*"дом" + 0.013*"идти" + 0.012*"никто" + 0.012*"утро" + 0.010*"последний" + 0.010*"настроение" + 0.010*"год" + 0.010*"работа" + 0.009*"неделя" + 0.007*"улица" + 0.007*"казаться" + 0.007*"прийтись" + 0.007*"нравиться" + 0.006*"снег" + 0.006*"прийти" +

In [16]:
print(lda.log_perplexity(corpus))

Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sc

-8.005086105397773


/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [17]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sc

Персплексия:  0.00033376074704742826


/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [18]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
